<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/fundamentos_logica/blob/main/4_fase_1_diagnostico_planificacion/0_formatos_clase/1_class_logica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/fundamentos_logica/blob/main/4_fase_1_diagnostico_planificacion/0_formatos_clase/1_class_logica.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>


# S3 — Tautologías, contradicciones y contingencias; equivalencias lógicas


**Clase:** Fundamentos de la Lógica — UdeA Campus Caucasia  
**Fecha:** Lunes 25/08/2025 — 4 h  
**Enfoques:** STEAMS + ABPP + TRRS (Duval)  
**Objetivo:** Identificar y clasificar proposiciones como tautologías, contradicciones o contingencias; traducir entre registros y experimentar en Jupyter.


In [2]:

# Entorno mínimo
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display, clear_output, Markdown
import ipywidgets as widgets

from sympy import symbols, Equivalent, simplify_logic, sympify
from sympy.logic.boolalg import truth_table

print("Entorno básico cargado. SymPy version:", __import__('sympy').__version__)


Entorno básico cargado. SymPy version: 1.14.0



## Funciones útiles: tabla de verdad y clasificación
Usa expresiones en sintaxis SymPy (ej.: `p & q`, `p | ~p`, `p >> q`, `Equivalent(p,q)`).


In [ ]:
import sympy as sp
from sympy.logic.boolalg import truth_table
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
from IPython.display import display, Markdown

# =========================
# Función tabla de verdad
# =========================
def tabla_verdad(expr, syms):
    rows = []
    header = [str(s) for s in syms] + [str(expr)]
    
    for assignment, value in truth_table(expr, syms):
        # Verificar si assignment ya es dict o no
        if isinstance(assignment, dict):
            assign_dict = assignment
        else:
            assign_dict = dict(assignment)

        row = [assign_dict.get(s, False) for s in syms] + [bool(value)]
        rows.append(row)
    
    return pd.DataFrame(rows, columns=header)

# =========================
# Clasificación
# =========================
def clasificar(expr, syms):
    df = tabla_verdad(expr, syms)
    col = df.iloc[:, -1]
    if col.all():
        return "Tautología", df
    elif (~col).all():
        return "Contradicción", df
    else:
        return "Contingencia", df

# =========================
# Animación de tabla
# =========================
def animar_tabla(df):
    fig, ax = plt.subplots()
    ax.axis('off')
    table = ax.table(cellText=df.values, colLabels=df.columns, loc='center', cellLoc='center')

    def update(i):
        for c in table.get_celld().values():
            c.set_facecolor("white")
        for j in range(df.shape[1]):
            table[(i+1, j)].set_facecolor("yellow")

    ani = FuncAnimation(fig, update, frames=len(df), repeat=False, interval=1000)
    plt.show()

# =========================
# Interfaz con ipywidgets
# =========================
def interfaz(expr_str):
    p, q = sp.symbols('p q')
    try:
        expr = sp.sympify(expr_str)
        clas, df = clasificar(expr, [p, q])
        display(Markdown(f"**Expresión:** ${sp.latex(expr)}$ → **{clas}**"))
        display(df)
        animar_tabla(df)
    except Exception as e:
        print("Error:", e)

# Entrada interactiva
entrada = widgets.Text(
    value="p | ~p",
    description="Expresión:",
    disabled=False
)

widgets.interact(interfaz, expr_str=entrada);



## Interfaz interactiva rápida
Escribe la expresión y los símbolos separados por comas. Ejemplo símbolos: `p,q` y expresión: `p & ~q`.


In [ ]:
import ipywidgets as widgets 

expr_input = widgets.Text(value='p & q', description='Expr:', layout=widgets.Layout(width='60%'))
syms_input = widgets.Text(value='p,q', description='Símbolos:', layout=widgets.Layout(width='40%'))
btn = widgets.Button(description='Evaluar', button_style='success')
out = widgets.Output()

def on_btn(b):
    with out:
        clear_output()
        txt = expr_input.value.strip()
        syms_list = [s.strip() for s in syms_input.value.split(',') if s.strip()!='']
        try:
            syms_dict = {s: symbols(s) for s in syms_list}
            expr_sym = sympify(txt, locals=syms_dict)
            syms_objs = [syms_dict[s] for s in syms_list]
            clas, df = clasificar(expr_sym, syms_objs)
            display(Markdown(f"**Fórmula:** ${expr_sym}$  \n**Clasificación:** **{clas}**"))
            display(df)
        except Exception as e:
            display(Markdown(f"**Error:** {e}"))

btn.on_click(on_btn)
display(widgets.HBox([expr_input, syms_input, btn]))
display(out)


Output()


## Animación ilustrativa
La siguiente animación muestra la evaluación de una expresión para todas las combinaciones de dos variables.


In [3]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sympy import symbols
from IPython.display import HTML, display, Markdown


def anim_demo(expr, syms):
    combos = [(0,0),(0,1),(1,0),(1,1)]
    fig, ax = plt.subplots(figsize=(4,2))
    ax.axis('off')
    txt = ax.text(0.5,0.6,'', ha='center', fontsize=12)
    res = ax.text(0.5,0.3,'', ha='center', fontsize=18)
    def update(i):
        a,b = combos[i%4]
        assign = {syms[0]: bool(a), syms[1]: bool(b)}
        val = bool(expr.subs(assign))
        txt.set_text(f'{syms[0]}={int(a)}, {syms[1]}={int(b)}')
        res.set_text('V' if val else 'F')
        res.set_bbox(dict(facecolor='lightgreen' if val else 'lightcoral'))
        return txt, res
    ani = FuncAnimation(fig, update, frames=4, interval=700, blit=True)
    plt.close(fig)
    return HTML(ani.to_jshtml())

# demo
display(Markdown("Animación demo: $(p \\land q) \\lor (\\lnot p \\land \\lnot q)$ (equivalencia)"))
display(anim_demo((p & q) | (~p & ~q), [p,q]))


Animación demo: $(p \land q) \lor (\lnot p \land \lnot q)$ (equivalencia)

NameError: name 'p' is not defined


## Actividades en clase (resumen)
- Mini-lab: probar 5 expresiones con la interfaz (pares).  
- Casio 991 LA cw: ejercicios de manipulación binaria y operaciones lógicas bit a bit.  
- Crocodile Clips: diseñar circuito de tautología y circuito de contradicción.  
- Mini-proyecto ABPP: regla de desempate en torneo → formalizar, probar y presentar.



## Tarea (20 ítems)
Cada ejercicio pide responder:  
1) ¿Por qué esta expresión representa el problema?  
2) ¿Qué pasaría si cambiamos este valor?  
3) ¿Hay otra manera de resolverlo?  
4) ¿Cómo justificas que tu resultado es razonable?

(La lista completa de expresiones se entregará en el aula y en el repositorio del curso.)



## Evaluación (ejemplo de quiz y parcial)
**Quiz (breve):** Formalizar la regla de puntos en un torneo y clasificar la expresión.  
**Parcial (estructura):** Tablas (40%), Equivalencias (30%), Aplicación (30%).



## Referencias breves (APA)
- Duval, R. (1993). Registros de representación semiótica y funcionamiento cognitivo del pensamiento matemático.  
- Wing, J. M. (2006). Computational thinking. Communications of the ACM, 49(3), 33–35.  
- Shannon, C. E. (1938). A symbolic analysis of relay and switching circuits.
